# Generate .sld XML styles for DEA Coastlines Geoserver

In [1]:
import matplotlib as mpl
from lxml import etree

def generate_xml_styles(start_year=1988, end_year=2023):

    # Interpolate inferno colour ramp between start and end date
    cmap = mpl.colormaps["inferno"]
    norm = mpl.colors.Normalize(vmin=start_year, vmax=end_year)

    # Convert to hex colours
    hex_values = {}
    for i in range(start_year, end_year):
        rgb = cmap(norm(i))
        hex_values[i] = mpl.colors.rgb2hex(rgb)
    hex_values[end_year] = "#ffffe1"  # Add custom end color

    # Base style
    base = """
    <StyledLayerDescriptor xmlns="http://www.opengis.net/sld" xmlns:ogc="http://www.opengis.net/ogc" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1.0" xsi:schemaLocation="http://www.opengis.net/sld http://schemas.opengis.net/sld/1.1.0/StyledLayerDescriptor.xsd" xmlns:se="http://www.opengis.net/se" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
        <NamedLayer>
            <se:Name>DEA Coastlines - shorelines annual{scale_name}</se:Name>
            <UserStyle>
                <se:Name>DEA Coastlines - shorelines annual{scale_name}</se:Name>
                <se:FeatureTypeStyle>
                </se:FeatureTypeStyle>
            </UserStyle>
        </NamedLayer>
    </StyledLayerDescriptor>
    """

    # Annual shoreline style
    annual_shoreline_rule = """
    <se:Rule xmlns:se="http://www.opengis.net/se">
        <se:Name>{year} ({quality} quality shorelines)</se:Name>
        <se:Description>
            <se:Title>{year} ({quality} quality shorelines)</se:Title>
        </se:Description>
        <ogc:Filter xmlns:ogc="http://www.opengis.net/ogc">
            <ogc:And>
                <ogc:Property{is_isnot}EqualTo>
                    <ogc:PropertyName>certainty</ogc:PropertyName>
                    <ogc:Literal>good</ogc:Literal>
                </ogc:Property{is_isnot}EqualTo>
                <ogc:PropertyIsEqualTo>
                    <ogc:PropertyName>year</ogc:PropertyName>
                    <ogc:Literal>{year}</ogc:Literal>
                </ogc:PropertyIsEqualTo>
            </ogc:And>
        </ogc:Filter>
        {scale}
        <se:LineSymbolizer>
            <se:Stroke>
                <se:SvgParameter name="stroke">{color}</se:SvgParameter>
                <se:SvgParameter name="stroke-linecap">square</se:SvgParameter>
                <se:SvgParameter name="stroke-linejoin">bevel</se:SvgParameter>
                {stroke}
            </se:Stroke>
        </se:LineSymbolizer>
    </se:Rule>
    """

    # Label style
    label_rule = """
    <se:Rule xmlns:ogc="http://www.opengis.net/ogc" xmlns:se="http://www.opengis.net/se">
        <se:MinScaleDenominator>0</se:MinScaleDenominator>
        <se:MaxScaleDenominator>10000</se:MaxScaleDenominator>
        <se:TextSymbolizer>
            <se:Label>
                <ogc:PropertyName>year</ogc:PropertyName>
            </se:Label>
            <se:Font>
                <se:SvgParameter name="font-family">SansSerif.plain</se:SvgParameter>
                <se:SvgParameter name="font-size">13</se:SvgParameter>
            </se:Font>
            <se:LabelPlacement>
                <se:LinePlacement>
                    <se:GeneralizeLine>true</se:GeneralizeLine>
                </se:LinePlacement>
            </se:LabelPlacement>
            <se:Halo>
                <se:Radius>2</se:Radius>
                <se:Fill>
                    <se:SvgParameter name="fill">#000000</se:SvgParameter>
                    <se:SvgParameter name="fill-opacity">0.477</se:SvgParameter>
                </se:Fill>
            </se:Halo>
            <se:Fill>
                <se:SvgParameter name="fill">#ffffff</se:SvgParameter>
            </se:Fill>
        </se:TextSymbolizer>
    </se:Rule>
    """

    # Set up good and bad quality linestrokes
    good_stroke = """
    <se:SvgParameter name="stroke-width">2</se:SvgParameter>
    """
    bad_stroke = """
    <se:SvgParameter name="stroke-width">1</se:SvgParameter>
    <se:SvgParameter name="stroke-dasharray">5 3.5</se:SvgParameter>
    """

    # Set up default and full zoom scales
    default_scale = """
    <se:MinScaleDenominator>0</se:MinScaleDenominator>
    <se:MaxScaleDenominator>25000</se:MaxScaleDenominator>
    """
    full_zoom = """
    <se:MinScaleDenominator>0</se:MinScaleDenominator>
    <se:MaxScaleDenominator>1500000</se:MaxScaleDenominator>
    """

    # Run twice for default and zoom zoom styles
    for scale in ["default", "full_zoom"]:
        
        # Update base naming
        base_renamed = base.format(scale_name="" if scale == "default" else " full zoom")

        # Initialise tree using base styling
        tree = etree.fromstring(base_renamed)

        # Loop through and add annual shoreline styles to tree
        feature_type_style = tree.find(".//se:FeatureTypeStyle", namespaces=tree.nsmap)
        for certainty in ["good", "bad"]:
            for year, color in hex_values.items():
                rule = annual_shoreline_rule.format(
                    year=year,
                    quality=certainty,
                    is_isnot="Is" if certainty == "good" else "IsNot",
                    color=color,
                    stroke=good_stroke if certainty == "good" else bad_stroke,
                    scale=default_scale if scale == "default" else full_zoom,
                )
                # Add into tree
                rule_xml = etree.fromstring(rule)
                feature_type_style.append(rule_xml)

        # Add label styling rule
        label_rule_xml = etree.fromstring(label_rule)
        feature_type_style.append(label_rule_xml)
        
        # Add sorting        
        sorting_rule_xml = etree.fromstring("""<se:VendorOption name="sortBy" xmlns:ogc="http://www.opengis.net/ogc" xmlns:se="http://www.opengis.net/se">year A</se:VendorOption>""")
        feature_type_style.append(sorting_rule_xml)        

        # Write it out
        name_dict = {
            "default": "shorelines_annual.sld",
            "full_zoom": "shorelines_annual_full_zoom.sld",
        }
        etree.indent(tree, '  ')  # required for proper indenting
        with open(name_dict[scale], "wb") as f:
            f.write(etree.tostring(tree, encoding="UTF-8", pretty_print=True, xml_declaration=True))

In [2]:
generate_xml_styles(start_year=1988, end_year=2023)